In [ ]:
import os
import io
import json
import time
import pickle
import base64
import random
import numpy as np
from tqdm import tqdm
from pathlib import Path
from PIL import Image
from vllm import LLM, SamplingParams
import re
from collections import Counter

In [ ]:
def seed_everything(seed):
    global SEED
    SEED = seed
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(322)

In [ ]:
def parse(text: str) -> str:
    pattern = r"FINAL\s*ANSWER\s*:\s*([A-Z]+)"
    match = re.search(pattern, text.strip(), re.IGNORECASE)
    if match:
        return match.group(1).upper()
    return ""

**Qwen 4B:**

1) Qwen/Qwen3-4B-Instruct-2507 (8 GB)
2) Qwen/Qwen3-4B-Thinking-2507 (8 GB)

**Qwen 8B:**

1) Qwen/Qwen3-8B (16 GB)

**For math:**

1) nvidia/OpenMath-Nemotron-7B (15 GB)

In [ ]:
MODEL_PATH = ""

In [ ]:
llm = LLM(
    model=MODEL_PATH,
    trust_remote_code=True,
    tensor_parallel_size=1,
    gpu_memory_utilization=0.9,
    max_model_len=4096,
    seed=SEED,
    # allowed_local_media_path=os.path.abspath('images'), # for VLM
)

In [ ]:
with open('input.pickle', "rb") as input_file:
    model_input = pickle.load(input_file)

In [ ]:
sampling_params_confident = SamplingParams(
    temperature=0.4,
    top_p=0.9,
    max_tokens=3600,
) # самые уверенные параметры
sampling_params_varied = SamplingParams(
    temperature=0.7,
    top_p=0.9,
    max_tokens=3600,
) # более разнообразные параметры

In [ ]:
BATCH_SIZE = 12
model_output = []

prompts = []
rids = []

os.makedirs("images", exist_ok=True)

# ======================== Формируем запросы ===========================
for row in tqdm(model_input):
    rid = row["rid"]
    question = row["question"]
    image = Image.open(io.BytesIO(row["image"]))
    image_path = f"images/{rid}.png"
    image.save(image_path)

    prompt = f"""
Choose all the correct options:\n{question}\n\n
Write a detailed explanation and the correct answer as a sequence of letters you chose and format it as 
FINAL ANSWER: <Answer> (example: FINAL ANSWER: AD). Rely only on the information present in the picture.
Check every fact very carefully and NEVER invent anything new or not related to the picture.
"""

    messages = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                # {"type": "image_url", "image_url": {"url": f"file://{os.path.abspath(image_path)}"}}, # FOR VLM
            ],
        }
    ]

    prompts.append(messages)
    rids.append(rid)


In [ ]:
# ======================== Инференс ===========================
for i in tqdm(range(0, len(prompts), BATCH_SIZE), desc="Inference"):
    batch_prompts = prompts[i:i + BATCH_SIZE]
    batch_rids = rids[i:i + BATCH_SIZE]

    all_answers = {rid: [] for rid in batch_rids}

    # --- Прогон 1 (уверенный) ---
    outputs1 = llm.chat(messages=batch_prompts, sampling_params=sampling_params_confident)
    for rid, output in zip(batch_rids, outputs1):
        text = output.outputs[0].text
        all_answers[rid].append(parse(text) or "B")

    # --- Прогоны 2 и 3 (вариативные) ---
    for _ in range(2):
        outputs = llm.chat(messages=batch_prompts, sampling_params=sampling_params_varied)
        for rid, output in zip(batch_rids, outputs):
            text = output.outputs[0].text
            all_answers[rid].append(parse(text) or "B")

    # --- Выбор финального ответа ---
    for rid in batch_rids:
        answers = [a for a in all_answers[rid] if a]
        unique = set(answers)
        if len(unique) == 3:
            # все три разные → берем 1-й (уверенный)
            final_answer = answers[0]
        else:
            # иначе мода
            final_answer = Counter(answers).most_common(1)[0][0]
        model_output.append({"rid": rid, "answer": final_answer})

In [ ]:
with open('output.json', 'w') as output_file:
    json.dump(model_output, output_file, ensure_ascii=False, indent=2)